Optimized Notebook – Performance Optimization & Exporting Data in WherobotsDB

This notebook extracts, transforms, and loads (ETL) large-scale GDELT event data into WherobotsDB before exporting it as an optimized GeoParquet dataset.

🔹 Workflow Overview¶
1️⃣ Create an optimized table in WherobotsDB (Iceberg format).
2️⃣ Organize data by GeoHash for spatial indexing.
3️⃣ Write the structured dataset as a GeoParquet file for efficient querying.

In [ ]:

from sedona.spark import SedonaContext

config = SedonaContext.builder() \
    .config(
        "spark.hadoop.fs.s3a.bucket.gdelt-open-data.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider") \
    .getOrCreate()

sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:


csv_path = 's3://gdelt-open-data/events/*.*.csv'

In [ ]:


csv_df = sedona.read.format("csv") \
    .option("delimiter", "\\t") \
    .load(csv_path)

In [ ]:
csv_df.count()

383958288

In [ ]:

import requests


response = requests.get('https://gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt')
response.raise_for_status()


header_line = response.text.splitlines()[0].strip()
headers = header_line.split('\t')

csv_df.toDF(*headers) zmienia domyślne nazwy kolumn na bardziej czytelne, zgodnie z listą headers.

In [ ]:

csv_df = csv_df.toDF(*headers)

In [ ]:
csv_df.printSchema()

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: stri

In [ ]:

csv_df.createOrReplaceTempView('csv_df')

In [ ]:
name = 'beatka'

🔸 CREATE OR REPLACE TABLE wherobots.{name}.gdelt AS
Tworzy nową tabelę w katalogu Spark SQL (tutaj: wherobots.{name}.gdelt). Jeśli tabela już istnieje, zostanie nadpisana.

name to zmienna w Pythonie — np. może być name = "example_user"

🔸 FROM csv_df
Źródło danych to DataFrame csv_df — czyli najpewniej dane z pliku CSV.

🔸 SELECT *, ST_SetSRID(ST_Point(ActionGeo_Long, ActionGeo_Lat), 4326) as geometry
SELECT *: wybiera wszystkie kolumny z csv_df

ST_Point(long, lat): tworzy punkt geometryczny z wartości długości i szerokości geograficznej

ST_SetSRID(..., 4326): ustawia układ współrzędnych (CRS) — tutaj EPSG:4326, czyli standardowy układ geograficzny WGS 84

as geometry: nadaje nazwę nowej kolumnie

In [ ]:
# Create the Havasu table and create a geometry
sedona.sql(f'''
CREATE OR REPLACE TABLE wherobots.{name}.gdelt AS
SELECT *,
ST_SetSRID(
    ST_Point(ActionGeo_Long, ActionGeo_Lat),
4326) as geometry
FROM csv_df
''')

DataFrame[]

In [ ]:
# Save the JSON file for the 4326 projection for the GeoParquet metadata

projjson = '''{
    "$schema": "https://proj.org/schemas/v0.7/projjson.schema.json",
    "type": "GeographicCRS",
    "name": "WGS 84",
    "datum_ensemble": {
        "name": "World Geodetic System 1984 ensemble",
        "members": [
            {
                "name": "World Geodetic System 1984 (Transit)",
                "id": {
                    "authority": "EPSG",
                    "code": 1166
                }
            },
            {
                "name": "World Geodetic System 1984 (G730)",
                "id": {
                    "authority": "EPSG",
                    "code": 1152
                }
            },
            {
                "name": "World Geodetic System 1984 (G873)",
                "id": {
                    "authority": "EPSG",
                    "code": 1153
                }
            },
            {
                "name": "World Geodetic System 1984 (G1150)",
                "id": {
                    "authority": "EPSG",
                    "code": 1154
                }
            },
            {
                "name": "World Geodetic System 1984 (G1674)",
                "id": {
                    "authority": "EPSG",
                    "code": 1155
                }
            },
            {
                "name": "World Geodetic System 1984 (G1762)",
                "id": {
                    "authority": "EPSG",
                    "code": 1156
                }
            },
            {
                "name": "World Geodetic System 1984 (G2139)",
                "id": {
                    "authority": "EPSG",
                    "code": 1309
                }
            }
        ],
        "ellipsoid": {
            "name": "WGS 84",
            "semi_major_axis": 6378137,
            "inverse_flattening": 298.257223563
        },
        "accuracy": "2.0",
        "id": {
            "authority": "EPSG",
            "code": 6326
        }
    },
    "coordinate_system": {
        "subtype": "ellipsoidal",
        "axis": [
            {
                "name": "Geodetic latitude",
                "abbreviation": "Lat",
                "direction": "north",
                "unit": "degree"
            },
            {
                "name": "Geodetic longitude",
                "abbreviation": "Lon",
                "direction": "east",
                "unit": "degree"
            }
        ]
    },
    "scope": "Horizontal component of 3D system.",
    "area": "World.",
    "bbox": {
        "south_latitude": -90,
        "west_longitude": -180,
        "north_latitude": 90,
        "east_longitude": 180
    },
    "id": {
        "authority": "EPSG",
        "code": 4326
    }
}'''

In [ ]:
import os
user_uri = os.getenv("USER_S3_PATH")

In [ ]:

gdelt = sedona.sql(f'''SELECT
*,
ST_GeoHash(geometry, 15) AS geohash,
struct(st_xmin(geometry) as xmin, st_ymin(geometry) as ymin, st_xmax(geometry) as xmax, st_ymax(geometry) as ymax) as bbox
FROM wherobots.{name}.gdelt''')

In [ ]:
%%time

gdelt.repartitionByRange(30, "geohash") \
    .sortWithinPartitions("geohash") \
    .drop("geohash15") \
    .write \
    .format("geoparquet") \
    .option("geoparquet.version", "1.1.0") \
    .option("geoparquet.covering", "bbox") \
    .option("geoparquet.crs", projjson) \
    .save(user_uri + "gdelt-snappy", mode='overwrite', compression='snappy')

25/04/16 21:43:55 ERROR TaskSetManager: Task 141 in stage 11.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o174.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 141 in stage 11.0 failed 4 times, most recent failure: Lost task 141.3 in stage 11.0 (TID 8438) (10.1.87.63 executor 1): java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:372)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:125)
	at java.base/java.io.BufferedOutputStream.implWrite(BufferedOutputStream.java:221)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:205)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:225)
	at net.jpountz.lz4.LZ4BlockOutputStr